## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kaitangata,NZ,-46.2817,169.8464,48.58,89,23,5.08,few clouds
1,1,Puerto Ayora,EC,-0.7393,-90.3518,76.96,83,37,9.89,scattered clouds
2,2,Sinnamary,GF,5.3833,-52.9500,83.17,67,83,11.36,broken clouds
3,3,Bengkulu,ID,-3.8004,102.2655,77.52,90,100,4.76,light rain
4,4,Ayan,RU,56.4500,138.1667,32.43,94,100,8.48,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,-0.7393,-90.3518,76.96,83,37,9.89,scattered clouds
2,2,Sinnamary,GF,5.3833,-52.9500,83.17,67,83,11.36,broken clouds
3,3,Bengkulu,ID,-3.8004,102.2655,77.52,90,100,4.76,light rain
6,6,Santa Rosa,PH,14.3122,121.1114,84.18,76,100,8.63,overcast clouds
11,11,Rikitea,PF,-23.1203,-134.9692,75.78,77,56,13.42,light rain
17,17,El Tarra,CO,8.5756,-73.0949,89.58,59,100,4.14,overcast clouds
29,29,Tabou,CI,4.4230,-7.3528,84.60,68,99,7.47,overcast clouds
30,30,Paramirim,BR,-13.4425,-42.2389,84.11,44,21,8.97,few clouds
31,31,Pimentel,PE,-6.8367,-79.9342,75.49,60,0,6.91,clear sky
32,32,Russell,US,32.3502,-85.1999,78.96,65,40,8.05,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                222
City                   222
Country                221
Lat                    222
Lng                    222
Max Temp               222
Humidity               222
Cloudiness             222
Wind Speed             222
Current Description    222
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()


City_ID                221
City                   221
Country                221
Lat                    221
Lng                    221
Max Temp               221
Humidity               221
Cloudiness             221
Wind Speed             221
Current Description    221
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,
2,Sinnamary,GF,83.17,broken clouds,5.3833,-52.9500,
3,Bengkulu,ID,77.52,light rain,-3.8004,102.2655,
6,Santa Rosa,PH,84.18,overcast clouds,14.3122,121.1114,
11,Rikitea,PF,75.78,light rain,-23.1203,-134.9692,
17,El Tarra,CO,89.58,overcast clouds,8.5756,-73.0949,
29,Tabou,CI,84.60,overcast clouds,4.4230,-7.3528,
30,Paramirim,BR,84.11,few clouds,-13.4425,-42.2389,
31,Pimentel,PE,75.49,clear sky,-6.8367,-79.9342,
32,Russell,US,78.96,scattered clouds,32.3502,-85.1999,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... Skipping")


Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [9]:
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].replace("", np.nan)


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()


City                   212
Country                212
Max Temp               212
Current Description    212
Lat                    212
Lng                    212
Hotel Name             212
dtype: int64

In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))